In [19]:
# !pip install gensim --user

In [1]:
# %load_ext line_profiler

In [2]:
import sys, os
# sys.path.insert(0, '/datasets/home/51/451/yuz530/.local/lib/python3.7/site-packages/')

In [3]:
!ls ./data/

A.npz B.npz P.npz


In [4]:
import numpy as np
from scipy import sparse

In [5]:
A = sparse.load_npz('./data/A.npz')
B = sparse.load_npz('./data/B.npz')
P = sparse.load_npz('./data/P.npz')
A_csr = A
A_csc = A.tocsc(copy=True)  # memory is cheap ;D

In [6]:
# ABPBA
def iter_neighbors():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
        api_p = np.random.choice(np.nonzero(P[:, api_bi])[0])
        api_bj = np.random.choice(np.nonzero(B[:, api_p])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_bj])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} api_{api_p} api_{api_bj} app_{app_j}'
        
it = iter_neighbors()

In [7]:
%lprun -f iter_neighbors a = [next(it) for i in range(10000)] 

UsageError: Line magic function `%lprun` not found.


In [8]:
f = open('test.cor', 'w')
for _ in range(10000):
    f.write(next(it) + '\n')
f.close()

In [9]:
!head test.cor

app_382 api_13362 api_13361 api_13359 api_13359 app_40
app_177 api_11532 api_280576 api_280578 api_730425 app_156
app_754 api_1374251 api_1374161 api_1374159 api_1374175 app_491
app_139 api_707022 api_707026 api_707031 api_706888 app_139
app_789 api_6014 api_31723 api_80147 api_80147 app_187
app_615 api_5996 api_1486215 api_1486204 api_1486196 app_770
app_542 api_17962 api_79 api_52518 api_1331134 app_458
app_63 api_184081 api_195 api_3774 api_305436 app_28
app_123 api_427637 api_390 api_12345 api_11503 app_421
app_364 api_799540 api_115 api_5272 api_174298 app_83


In [17]:
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'test.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [18]:
%%time

import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

CPU times: user 8 s, sys: 2.36 s, total: 10.4 s
Wall time: 8.32 s


In [22]:
for i, word in enumerate(model.wv.vocab):
    print(word)

app_
api_


In [12]:
gensim.models.Word2Vec.FAST_VERSION

AttributeError: type object 'Word2Vec' has no attribute 'FAST_VERSION'

In [13]:
!pip install Cython

In [14]:
from gensim.models.word2vec import FAST_VERSION

In [15]:
FAST_VERSION

-1